# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, randint
import joblib

## Dataset

### Overview

In this project, we are going to classify the progressive muscle weakness and disease with the use of SKLearn Classifier and AutoML.

The dataset is custom data curated from one of the health institution.

The dataset contains 3 categories:
1. Control
    
    Indicates whether muscle weakness is in control or not.
    
    
2. SPG4
    
    **S pastic paraplegia 4 (SPG4)** is the most common type of hereditary spastic paraplegia (HSP) inherited in an autosomal dominant manner. Disease onset ranges from infancy to older adulthood. SPG4 is characterized by slowly progressive muscle weakness and spasticity (stiff or rigid muscles) in the lower half of the body. In rare cases, individuals may have a more complex form with seizures, ataxia, and dementia. SPG4 is caused by mutations in the SPAST gene. Severity of symptoms usually worsens over time, however some individuals remain mildly affected throughout their lives. Medications, such as antispastic drugs and physical therapy may aid in stretching spastic muscles and preventing contractures (fixed tightening of muscles) 
    
    Source : https://rarediseases.info.nih.gov/diseases/4925/spastic-paraplegia-4
    
    
3. Disease
   
   Indicates that the muscle weakness is there due to some desease.


In [ ]:
ws = Workspace.from_config()
experiment_name = 'mwd-hyperdrive'

experiment=Experiment(ws, experiment_name)

In [ ]:

try:
    cpu_cluster = ComputeTarget(workspace=ws, name="mwd-compute")
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,"mwd-compute", compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

We have used RandomForestClassifier algorithm as we are solving the "classification" problem. RandomForest is consist of multiple DecisionTrees algorithm which play an important part in the output of the RandomForest.

We have applied an eary stopping policy to stop the training process after it starts to degrading the accuracy with increased iteration count. 

For the hyperparam tuninng, we have used RandomParameterSampling where we used random depth upto 50 for the model and also given some choices for the number of estimators param of the model.

The primary metric for our algo is "Accuracy". The hyperdrive willl try to improve the accuracy by giving provided choices of hyperparams. We also specified max_total_runs param which denotes as a param to run number of iteration to complete the hyper drive for training the model. Also we have provided "max_concurrent_runs" option to run the iterations of hyperdrive paralley so the process gets complete faster.

In [ ]:
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "d": randint(50),
    "n": choice(50,100,150,200,250)
});

#TODO: Create your estimator and hyperdrive config
env = Environment.from_pip_requirements(name='venv', file_path='./requirements.txt', pip_version='21.0.1')

estimator = ScriptRunConfig(source_directory = ".", compute_target = cpu_cluster, script = 'train.py', environment = env)

hyperdrive_run_config = HyperDriveConfig(run_config = estimator, 
                             hyperparameter_sampling = param_sampling,
                             policy = early_termination_policy,
                             primary_metric_name = 'Accuracy', 
                             primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs = 20,
                             max_concurrent_runs = 5)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run, best_model = hyperdrive_run.get_output()

best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

In [ ]:
print(best_run.get_details()['runId'])
best_run.get_details()['runDefinition']['arguments']

In [ ]:
#TODO: Save the best model
best_run.register_model(experiment_name+"-best-model","outputs/"+experiment_name+"-hyperdrive-model.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

We already got 97.65 accuracy through XGBoostClassifier using AutoML so we are not deploying this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service